## COCO Evaluating with Ground Truths

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import os
import mlflow

In [ ]:
# Setting MLFLOW Configurations
os.environ['MLFLOW_TRACKING_USERNAME'] = 'thesisuist'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'cd6a7567bea2de310e1d52ac2f35483fca241e95'

In [ ]:
# Setting the tracking URI: DagsHub MLFlow repository
mlflow.set_tracking_uri('https://dagshub.com/thesisuist/reversed_final.mlflow')

# Getting the Run ID
run_id = '6b8969424815419ab706b8aff4de27a8'

In [ ]:
artifacts = mlflow.tracking.MlflowClient().list_artifacts(run_id)

In [ ]:
# Printing the artifact files of the chosen experiment
artifacts

[<FileInfo: file_size=None, is_dir=True, path='after_train_config_file'>,
 <FileInfo: file_size=None, is_dir=True, path='after_training_config_file'>,
 <FileInfo: file_size=132420, is_dir=False, path='events.out.tfevents.1715871614.gorina9.4186487.0'>,
 <FileInfo: file_size=132420, is_dir=False, path='events.out.tfevents.1715872291.gorina8.1127786.0'>,
 <FileInfo: file_size=131971, is_dir=False, path='events.out.tfevents.1716708368.gorina8.4100335.0'>,
 <FileInfo: file_size=15, is_dir=False, path='last_checkpoint'>,
 <FileInfo: file_size=44076, is_dir=False, path='loss_curve.png'>,
 <FileInfo: file_size=262234, is_dir=False, path='metrics.json'>,
 <FileInfo: file_size=6130, is_dir=False, path='model-config.yaml'>,
 <FileInfo: file_size=330107348, is_dir=False, path='model_0000499.pth'>,
 <FileInfo: file_size=330107348, is_dir=False, path='model_0000999.pth'>,
 <FileInfo: file_size=330107348, is_dir=False, path='model_0001499.pth'>,
 <FileInfo: file_size=330107348, is_dir=False, path='m

In [ ]:
mlflow.tracking.MlflowClient().download_artifacts(run_id, 'test-set-evaluation/coco_instances_results.json', 'artifacts')

'/mnt/beegfs/home/aashishk/for_github/notebooks/artifacts/test-set-evaluation/coco_instances_results.json'

In [ ]:
# Getting the groundtruth test file and the output coco results file
cocoGt = COCO("../data/Phase II/test.json")
cocoDt = cocoGt.loadRes("artifacts/test-set-evaluation/coco_instances_results.json")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!


#### Evaluating and Summarizing

In [ ]:
cocoEval = COCOeval(cocoGt, cocoDt, "bbox")
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.21s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.729
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.912
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.822
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.702
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.734
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.433
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.820
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.820
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100

In [ ]:
# Function to calculate IoU
def calculate_iou(boxA, boxB):
    """
    Calculates the Intersection over Union (IoU) of two bounding boxes.

    Parameters:
    - boxA (tuple): A tuple (x, y, width, height) representing the first bounding box.
    - boxB (tuple): A tuple (x, y, width, height) representing the second bounding box.

    Returns:
    - float: The Intersection over Union (IoU) between the two bounding boxes.
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    iArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    iou = iArea / float(boxAArea + boxBArea - iArea)
    return iou

In [ ]:
# Defining Threshold
iou_threshold = 0.5

# Defaults; Initializing the variables
tp = 0
fp = 0
fn = 0

# Fetching Categories / Classes
categories = cocoGt.loadCats(cocoGt.getCatIds())

for catId in cocoGt.getCatIds():
    gt_ann_ids = cocoGt.getAnnIds(catIds=[catId])
    dt_ann_ids = cocoDt.getAnnIds(catIds=[catId])

    gt_anns = cocoGt.loadAnns(gt_ann_ids)
    dt_anns = cocoDt.loadAnns(dt_ann_ids)

    matched_gt_ids = set()
    matched_dt_ids = set()

    for gt in gt_anns:
        best_iou = 0
        best_dt_id = None
        for dt in dt_anns:
            iou = calculate_iou(dt['bbox'], gt['bbox'])
            if iou > best_iou:
                best_iou = iou
                best_dt_id = dt['id']
        
        if best_iou >= iou_threshold:
            matched_gt_ids.add(gt['id'])
            matched_dt_ids.add(best_dt_id)
    
    tp = len(matched_gt_ids)
    fp = len(dt_anns) - len(matched_dt_ids)
    fn = len(gt_anns) - len(matched_gt_ids)

    className = cocoGt.loadCats(catId)[0]['name']
    print("For " + className)
    print(f'True Positives (TP): {tp}')
    print(f'False Positives (FP): {fp}')
    print(f'False Negatives (FN): {fn}')
    print("==============================")


For caption
True Positives (TP): 185
False Positives (FP): 34
False Negatives (FN): 7
For table
True Positives (TP): 192
False Positives (FP): 15
False Negatives (FN): 2
For table_caption
True Positives (TP): 190
False Positives (FP): 36
False Negatives (FN): 0
